## Download & Analyze Darwin Core Archive(s)
Originally based on script from Matt Biddle
https://github.com/MathewBiddle/sandbox/blob/main/notebooks/query_obis_api.ipynb

Use this notebook to do some analysis and visualize your DwC archive.
Your DwC archive must be accessible on an IPT server.

In [3]:
# Import requests and set the OBIS API base URL. 
import requests
import json
import pandas as pd
import urllib

# Convenience function to pretty print JSON objects
def print_json(myjson):
    print(json.dumps(
        myjson,
        sort_keys=True,
        indent=4,
        separators=(',', ': ')
    ))

# Initialize the base URL for OBIS. This variable will be used for every API call
OBIS_URL = "https://api.obis.org/v3"

# The "name" of the IPT node we want to inspect 
MY_NODE_NAME = 'OBIS Canada'

# The OBIS ID assigned to the dataset we want to inspect
# You can get this from the OBIS URL of your dataset: https://obis.org/dataset/{OBIS dataset ID here} 
DATASET_ID = '24e96d02-8909-4431-bc61-8cf8eadc9b7a'

In [7]:
# === get the DwC archive
# TODO: def get_dwc_dataset(OBIS_URL, MY_NODE_NAME, DATASET_ID):
# figure out the node ID
req = requests.get(f'{OBIS_URL}/node')
nodes_json = req.json()
df_nodes = pd.DataFrame(nodes_json['results'])
nodeID = df_nodes.loc[df_nodes['name']==MY_NODE_NAME,'id'].tolist()[0]

# get metadata from all datasets @ MY_NODE_NAME
req = requests.get(f'{OBIS_URL}/dataset?nodeid={nodeID}')
datasets = req.json()['results']

# get the dataset we care about:
for ds in datasets:
    # print(ds['id'])
    if ds['id'] == DATASET_ID:
        dataset = ds
        print(f"found datset {dataset['title']} \n\tw/ ID = {dataset['id']}")
        break  # return dataset
else:
    raise ValueError(f"No dataset found with ID[{DATASET_ID}")

# dataset = get_dwc_dataset(OBIS_URL, MY_NODE_NAME, DATASET_ID)
# display(dataset)

found datset Trawl Catch and Species Abundance from the 2019 Gulf of Alaska International Year of the Salmon Expedition 
	w/ ID = 24e96d02-8909-4431-bc61-8cf8eadc9b7a


In [4]:
# grab metadata about the dataset

from bs4 import BeautifulSoup

columns = ['title','url','size_raw','size_MB']

df = pd.DataFrame(
        columns=columns
    )

print(dataset['title'])
print(dataset['url'])
html_text = requests.get(dataset['url']).text
soup = BeautifulSoup(html_text, 'html.parser')

size_raw = soup.find('td').text.split('(')[1].split(')')[0]
size = float(size_raw.split(" ")[0].replace(",",""))
size_unit = size_raw.split(" ")[1]

#convert sizes to MB
if size_unit == 'KB':
    size = size*0.001
elif size_unit == 'MB':
    size = size

df_init = pd.DataFrame(
            {"title": dataset['title'],
             "url": dataset['url'],
             "size_raw": size_raw,
             "size_MB": size,
             },
          index=[1])

df = pd.concat([df, df_init], ignore_index=True)

Trawl Catch and Species Abundance from the 2019 Gulf of Alaska International Year of the Salmon Expedition
http://ipt.iobis.org/obiscanada/resource?r=trawl-catch-and-species-abundance-from-the-2019-gulf-of-alaska-international-year-of-the-salmon-expedition


In [5]:
## Download the Darwin Core Archive package
# download the [DwC-A](https://github.com/gbif/ipt/wiki/DwCAHowToGuide#what-is-darwin-core-archive-dwc-a) zip package. 
#
# To do that:
# 1. Collect the DwC-A zip url by parsing the **IPT** dataset html page, looking for the **Data as a DwC-A file** `download` link.  
# 1. We download the zip package to the file `OBIS_data/{dataset short name}.zip` (eg. `OBIS_data/habsos.zip`) 
                                          
import os

LOCAL_OBIS_DATA_PATH = "./OBIS_data"

# create the data folder if it doesn't already exist
os.makedirs(LOCAL_OBIS_DATA_PATH, exist_ok=True)

url = df['url'][0]
# print(f"DwC Archive URL: {url}")
html_text = requests.get(url).text
soup = BeautifulSoup(html_text, 'html.parser')
size_raw = soup.find('td')

zip_download = size_raw.find('a').get('href')

vers = zip_download.split("=")[-1]
name = zip_download.split("=")[-2].replace("&v","")

fname = LOCAL_OBIS_DATA_PATH + '/' + name + '_v' + vers + '.zip'

print('Downloading ' + zip_download)
print('Downloading to ' + fname)
urllib.request.urlretrieve(zip_download, fname)
print('Complete.')

Complete.


Use the [darwin core python reader package](https://python-dwca-reader.readthedocs.io/en/latest/index.html) to print out some metadata about the DwC-A package.

In [6]:
from dwca.read import DwCAReader

with DwCAReader(fname) as dwca:
    print(dwca.archive_path)
    root = dwca.metadata
    node = root.find('.//westBoundingCoordinate')
    print('%s: %s' % (node.tag, node.text))

./OBIS_data/trawl-catch-and-species-abundance-from-the-2019-gulf-of-alaska-international-year-of-the-salmon-expedition_v1.1.zip
westBoundingCoordinate: -147.527


Now lets do some automated ingest of all the data:
1. For each zip package
   1. Read the core file into a Pandas DataFrame.
   1. Concatenate all the core data into one large data frame.
   1. Print out some useful information as each package is processed.

In [7]:
from dwca.read import DwCAReader
from dwca.darwincore.utils import qualname as qn
import pandas as pd
import os

core_df = pd.DataFrame()
# occurrence only = OBIS_data/wod_2009.zip
# event = OBIS_data/ambon_cetaceans_2015.zip
for obis_zip in os.listdir(LOCAL_OBIS_DATA_PATH):
    if not obis_zip == 'unzipped':
        with DwCAReader(LOCAL_OBIS_DATA_PATH + '/'+obis_zip) as dwca:
            #eml = dwca.metadata
            print("\nReading: %s" % dwca.archive_path)
            print("Core type is: %s" % dwca.descriptor.core.type)
            print("Core data file is: %s" % dwca.descriptor.core.file_location)
            for ex in dwca.descriptor.extensions:
                print('Extensions: ',ex.type)
                
            df_init = dwca.pd_read(dwca.core_file_location, parse_dates = True)
            df_init['zip_name'] = obis_zip
            
            core_df = pd.concat(
                [core_df, df_init], 
                axis = 0, 
                ignore_index = True)


Reading: ./OBIS_data/trawl-catch-and-species-abundance-from-the-2019-gulf-of-alaska-international-year-of-the-salmon-expedition_v1.1.zip
Core type is: http://rs.tdwg.org/dwc/terms/Event
Core data file is: event.txt
Extensions:  http://rs.tdwg.org/dwc/terms/ResourceRelationship
Extensions:  http://rs.iobis.org/obis/terms/ExtendedMeasurementOrFact
Extensions:  http://rs.tdwg.org/dwc/terms/Occurrence


In [8]:
# === print some basic stuff about the package
print(f"filepath {dwca.archive_path}")

root = dwca.metadata

print("roles:")
for child in root.findall('.//role'):
    print("\t", child.tag, ":", child.text)
    
print("\n(# rows, # cols):")
print(core_df.shape)

print("\ncolumns in core file: ")
print(core_df.columns.to_list())

filepath ./OBIS_data/trawl-catch-and-species-abundance-from-the-2019-gulf-of-alaska-international-year-of-the-salmon-expedition_v1.1.zip
roles:
	 role : distributor
	 role : distributor
	 role : publisher

(# rows, # cols):
(2613, 18)

columns in core file: 
['id', 'type', 'eventID', 'parentEventID', 'samplingProtocol', 'eventDate', 'year', 'month', 'day', 'locationID', 'locality', 'minimumDepthInMeters', 'maximumDepthInMeters', 'decimalLatitude', 'decimalLongitude', 'coordinateUncertaintyInMeters', 'footprintWKT', 'zip_name']


In [70]:
"""
# check the eventDate column values
"""
import pandas as pd
import numpy

# === try reading "basic" datetimes
print("looking for datetimes...")
basic_datetimes_df = pd.to_datetime(core_df['eventDate'], errors='coerce')
print(f"\t  valid basic datetimes: {basic_datetimes_df.count()}")
print(f"\tinvalid basic datetimes: {len(basic_datetimes_df) - basic_datetimes_df.count()}")

# === try reading timedeltas
# NOTE: can't use pd.to_timedelta here b/c we aren't looking for timedeltas only,
#       we want one of:
#           * start_dt + timedelta (not yet implemented)
#           * timedelta + end_dt (not yet implemented)
#           * start_dt + end_dt (ref http://dotat.at/tmp/ISO_8601-2004_E.pdf section 4.4.4.1)
print("looking for {start}/{end} timedeltas...")
# split start & end assuming seperated by `/`
start_end_timedeltas_str_df = core_df['eventDate'].str.split('/', expand=True)
n_td2_cols = start_end_timedeltas_str_df.shape[1]
if n_td2_cols == 2:
    print("\t`eventDate`s *might* be ISO8601 timedeltas identified by start & end")
    starts = pd.to_datetime(start_end_timedeltas_str_df[0], errors='coerce')
    ends = pd.to_datetime(start_end_timedeltas_str_df[1], errors='coerce')
    start_end_timedeltas_df = ends - starts
    # set datetime to exact middle of time span given
    start_end_datetimes_df = starts + start_end_timedeltas_df/2
elif n_td2_cols == 1:
    print("\tno `/` found in eventDate column")
elif n_td2_cols > 2:
    print(
        "\tmultiple `/` found in eventDate column;"
        " could not attempt parsing as ISO8601 start & end timedeltas."
    )
else:
    raise AssertionError(
        "Somehow eventDate split into 0 or less columns."
        " This should never happen."
    )
print(f"\t  valid start/end datetimes: {start_end_datetimes_df.count()}")
print(f"\tinvalid start/end datetimes: {len(start_end_datetimes_df) - start_end_datetimes_df.count()}")    

# === combine all datetime dataframes to get one master df
def df_combine_method(s1, s2):
    """
    Combines two datetime values s1 & s2 into one value.
    
    Assumes s1 == s2 or one of (s1,s2) are NaT (Not a Time).
    Raises if s1 & s2 are different valid dates.
    """
    # print(f"{s1} ({type(s1)}) + {s2} ({type(s2)})")
    s1 = numpy.datetime64(s1)
    s2 = numpy.datetime64(s2)
    if s1 == s2 or (numpy.isnat(s1) and numpy.isnat(s2)):
        return s1
    elif not numpy.isnat(s1) and not numpy.isnat(s2):
        raise ValueError("got two different datetimes from two different ways of parsing")
    elif not numpy.isnat(s1) and numpy.isnat(s2):
        return s1
    elif numpy.isnat(s1) and not numpy.isnat(s2):
        return s2
    else:
        raise AssertionError(
            f"Both are not NaT, not equal, and not different."
            " This should never happen."
            " {s1} ({type(s1)}) + {s2} ({type(s2)})"
        )

# ~pd.isna is used to select only valid (non 'NaT') values from each
combined_datetimes_df = basic_datetimes_df[~pd.isna(basic_datetimes_df)].combine(
    start_end_datetimes_df[~pd.isna(start_end_datetimes_df)], 
    df_combine_method
)

print("\n all total ISO8601 compliant `eventDate` values:")
print(f"\t  valid: {combined_datetimes_df.count()}")
print(f"\tinvalid: {len(core_df) - len(combined_datetimes_df)}")


print(f"\ndatetime range: {combined_datetimes_df.min()} / {combined_datetimes_df.max()}")

print("\nlist of invalid eventDate values:")
invalid_values = core_df['eventDate'][na_datetimes_df].unique()
print('\t', invalid_values)

if len(invalid_values) > 0:
    raise ValueError(
        f"WARN: {len(invalid_values)} invalid `eventDate` values found!"
        " `eventDate` should be an ISO8601-formatted datetime."
        " nan is not a valid value."
        " Please fix this and reupload to IPT."
    )

looking for datetimes...
	  valid basic datetimes: 0
	invalid basic datetimes: 2613
looking for {start}/{end} timedeltas...
	`eventDate`s *might* be ISO8601 timedeltas identified by start & end
	  valid start/end datetimes: 2547
	invalid start/end datetimes: 66

 all total ISO8601 compliant `eventDate` values:
	  valid: 2547
	invalid: 66

datetime range: 2019-02-19 00:32:00 / 2019-03-16 14:39:00

list of invalid eventDate values:
	 [nan]


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:55: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


ValueError: WARN: 1 invalid `eventDate` values found! `eventDate` should be an ISO8601-formatted datetime. nan is not a valid value. Please fix this and reupload to IPT.